<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_12_31_Anthropic_LLM_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to implement streaming Anthropic LLM completions using Graphlit for RAG retrieval.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

Assign this property as Colab secret: ANTHROPIC_API_KEY.


---

Install Graphlit Python client SDK

In [8]:
!pip install --upgrade graphlit-client

Install Anthropic Python SDK

In [9]:
!pip install --upgrade anthropic

Initialize Graphlit

In [10]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Initialize Anthropic

In [30]:
from anthropic import Anthropic

os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))


Define Graphlit helper functions

In [31]:
from typing import List, Optional

async def ingest_uri(uri: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_anthropic_specification(model: enums.AnthropicModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Anthropic [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=model,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_conversation(conversation_id: str):
    if graphlit.client is None:
        return;

    if conversation_id is not None:
        _ = await graphlit.client.delete_conversation(conversation_id)

async def format_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.format_conversation(prompt, conversation_id)

        return response.format_conversation.message.message if response.format_conversation is not None and response.format_conversation.message is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def complete_conversation(conversation_id: str, completion: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.complete_conversation(completion, conversation_id)

        return response.complete_conversation.message.message if response.complete_conversation is not None and response.complete_conversation.message is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def get_conversation(conversation_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.get_conversation(conversation_id)

        return response.conversation
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


In [44]:
from typing import Callable

def stream_completion(prompt: str, model: str, callback: Callable[[str], None]) -> str:
    stream = client.messages.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        max_tokens=4096,
        stream=True
    )

    completion = ""

    for event in stream:
        if event is not None and event.type == "content_block_delta":
            callback(event.delta.text)

            completion += event.delta.text

    return completion


Execute Graphlit example

In [14]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing contents, conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()
await delete_all_contents()

print('Deleted all contents, conversations and specifications.')

content_id = await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Unstructured%20Data%20is%20Dark%20Data%20Podcast.mp3")

if content_id is not None:
    print(f'Ingested content [{content_id}]:')

Deleted all contents, conversations and specifications.
Ingested content [2d0777a5-9984-4f2e-b974-a9eae7b65942]:


In [15]:
    # Specify the RAG prompt
    prompt = "In 3-5 detailed paragraphs, explain unstructured data and its usefulness for knowledge capture and retrieval."

Create Anthropic Claude Sonnet 3.5 specification.

In [45]:
    model = "claude-3-5-sonnet-20241022"

    specification_id = await create_anthropic_specification(enums.AnthropicModels.CLAUDE_3_5_SONNET_20241022)

    if specification_id is not None:
        print(f'Created specification [{specification_id}].')

        conversation_id = await create_conversation(specification_id)

        if conversation_id is not None:
            print(f'Created conversation [{conversation_id}].')

            # NOTE: returns LLM-ready formatted prompt from RAG pipeline
            message = await format_conversation(conversation_id, prompt)

            if message is not None:
                # NOTE: uncomment to see formatted LLM prompt

                #print(f'Formatted LLM prompt:')
                #print(message)
                #print()

                print(f'Streaming completion:')

                completion = stream_completion(message, model, lambda delta: print(delta, end=''))

                if completion is not None:
                    # NOTE: stores completion back into conversation
                    await complete_conversation(conversation_id, completion)

            conversation = await get_conversation(conversation_id)

            if conversation is not None:
                display(Markdown(f'### Conversation [{conversation.id}]:'))

                if conversation.messages is not None:
                    for message in conversation.messages:
                        if message is not None:
                            display(Markdown(f'**{message.role}:**\n{message.message}'))

                print()

            await delete_conversation(conversation_id)

Created specification [7fed5400-da89-4087-acc0-aa6e17db5431].
Created conversation [6e216851-edc4-4b5a-913c-4ec37f33584f].
Streaming completion:
Unstructured data encompasses a broad range of content types including images, audio, 3D geometry, point clouds, documents and emails - essentially any file-based data that isn't neatly organized in traditional database tables. While these files actually do have defined structures and formats, the term "unstructured" refers more to how the meaningful content within them requires additional processing and analysis to extract useful information and insights.

The value of unstructured data can be understood through the concept of layered metadata extraction. First-order metadata includes basic file attributes like timestamps and headers. Second-order metadata comes from analyzing the actual content, such as performing object detection on images or transcribing audio. Third-order metadata involves making higher-level inferences and connections, l

### Conversation [6e216851-edc4-4b5a-913c-4ec37f33584f]:

**USER:**
In 3-5 detailed paragraphs, explain unstructured data and its usefulness for knowledge capture and retrieval.

**ASSISTANT:**
Unstructured data encompasses a broad range of content types including images, audio, 3D geometry, point clouds, documents and emails - essentially any file-based data that isn't neatly organized in traditional database tables. While these files actually do have defined structures and formats, the term "unstructured" refers more to how the meaningful content within them requires additional processing and analysis to extract useful information and insights.

The value of unstructured data can be understood through the concept of layered metadata extraction. First-order metadata includes basic file attributes like timestamps and headers. Second-order metadata comes from analyzing the actual content, such as performing object detection on images or transcribing audio. Third-order metadata involves making higher-level inferences and connections, like linking detected objects to real-world assets in other systems or identifying relationships between different pieces of content. This layered approach allows increasingly sophisticated knowledge capture from raw data.

A major challenge with unstructured data is that it often becomes "dark data" over time - valuable information that gets archived but becomes difficult to access or analyze effectively. Modern knowledge graph approaches help address this by creating dynamic connections between different pieces of content and their extracted insights. Rather than just searching file names or doing basic text searches, this enables semantic searching across relationships and patterns in the data. For example, drone imagery can be automatically analyzed to detect buildings and infrastructure, then linked to relevant maintenance documents and inspection records to provide rich contextual information.

The end goal is to transform raw unstructured content into discoverable, analyzable knowledge that provides real business value. This could mean identifying trends over time, automatically alerting on important observations, or enabling powerful semantic search across previously disconnected information. The key is having systems that can not only extract insights from individual pieces of content, but also create meaningful connections between different data sources to build a comprehensive knowledge network that grows more valuable over time through continuous enrichment and analysis.